In [1]:
from nltk.stem.porter import *
import csv
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [2]:
training_path = r"C:\Users\Uly\Desktop\Desktop\UCL\IRDM\data\joined_data.csv"
stemmer = PorterStemmer()

In [3]:
def str_stem(s):
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

In [14]:
#relevance
def feat_1(rows:list, feat_dict):
    feat_dict.update({'relevance:':float(rows[0][0])})

#search term
def feat_2(rows:list, feat_dict):
    feat_dict.update({'search term:': rows[0][2]})

#product description - bag of words
def feat_3(rows:list, feat_dict):
    bag_of_words = defaultdict(int)    
    stemmed = [str_stem(w) for w in rows[0][4].split(' ')]
    for word in stemmed:
        bag_of_words[word] += 1
        
    temp_dict = {}
    for k,v in bag_of_words.items():
        temp_dict['description:' + k] = v
    
    feat_dict.update(temp_dict)

#attributes
def feat_4(rows:list, feat_dict):
    temp_dict = {}
    for row in rows:
        temp_dict[row[5]] = row[6]
        
    feat_dict.update(temp_dict)

In [15]:
#load data
def load_data(filepath):  
    data = []
    id = -1
    event = []
    
    with open(filepath, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')  
        next(reader)

        for row in reader:
            #Accumulate all the rows with the same ID
            if int(row[0]) != id:
                #process event and add it to the events list
                if id == -1:
                    id = int(row[0])
                    event.append(row)
                if len(event) > 0:
                    instance = {}
                    feat_1(event,instance)
                    feat_2(event,instance)
                    feat_3(event,instance)
                    feat_4(event,instance)
                    data.append(instance)
                    len(data)
                    event.clear()
                    id = int(row[0])
                    event.append(row)
            else:
                event.add(row)
            
    return data

In [16]:
def vectorize_data(data:list):
    vectorizer = DictVectorizer()
    event_x = vectorizer.transform(data)
    return event_x

In [17]:
print('Loading data...')
training_data = load_data(training_path)

Loading data...


NameError: name 'strNum' is not defined

In [35]:
len(training_data)

0

In [23]:
print('Vectorizer')
vectorized_data = vectorize_data(training_data)

Vectorizer


ValueError: Sample sequence X is empty.

In [ ]:
# converts labels into integers, and vice versa, needed by scikit-learn.
label_encoder = LabelEncoder()

# encodes feature dictionaries as numpy vectors, needed by scikit-learn.
vectorizer = DictVectorizer()

args = {"feat_1": True, "feat_2": True,"feat_3": True,"feat_4": True,"feat_5": True,"feat_6": True, \
"feat_7": True,"feat_8": True, "feat_9": True,"feat_10": False, "feat_11": True, "feat_12": True, "feat_13": True}

def event_feat(event):
    """
    This feature function returns a dictionary representation of the event candidate. You can improve the model 
    by improving this feature function.
    Args:
        event: the `EventCandidate` object to produce a feature dictionary for.
    Returns:
        a dictionary with feature keys/indices mapped to feature counts.

    result = defaultdict(float)
    result['trigger_word=' + event.sent.tokens[event.trigger_index]['word']] += 1.0
    return result
    """
    result = {}
    #Feature 1: trigger word stem

    return result
    

def predict_event_labels(event_candidates):
    event_x = vectorizer.transform([event_feat(e) for e in event_candidates])
    event_y = label_encoder.inverse_transform(lr.predict(event_x))
    return event_y